In [1]:
import requests
import json
import pandas as pd
import datetime
#from dateutil.tz import gettz
import os

In [2]:
#request할 역의 list를 담은 file load
stationInfo = pd.read_csv("/content/drive/MyDrive/statnNm/total_subway_line.csv") #You should check your file path
stationInfo #checking 

,역명
0,서울
1,영등포
2,수원
3,청량리
4,용산
...,...
145,신설동
146,용답
147,용두
148,신답


In [3]:
temp = stationInfo['역명']

In [4]:
def getJson(url): #request function
  response = requests.get(url)
  #response가 제대로 왔는지 checking
  if response.status_code is not 200:
    print("response Error")
    exit()
  
  return response.json()


In [5]:
def saveData(df, statnNm): #save function
  fileName = "/content/drive/MyDrive/real-time-data/" + statnNm.replace('/', '') + ".csv"
  if not os.path.exists(fileName):
    df.to_csv(fileName, mode='w', index=False, encoding='utf-8-sig')
  else:
    df.to_csv(fileName, mode='a', index=False, encoding='utf-8-sig', header=False)

In [6]:
def main():
  ServiceKeyList = ["524e6361756a75613739536b6a7a54"] #서비스 키 list
  KEY = '/' + ServiceKeyList[0] 
  TYPE = '/' + "json"
  SERVICE = '/' + "realtimeStationArrival"
  START_INDEX = '/' + '0'
  END_INDEX = '/' + '10'

  for idx in range(len(temp)):
    #search할 target station을 list로 만들어서 크롤링 할 것
    #if list가 큰 경우 file로 만들어서 load/save 할 것 
    statnNm = '/' + temp[idx] 

    url = "http://swopenAPI.seoul.go.kr/api/subway" + KEY + TYPE + SERVICE + START_INDEX + END_INDEX + statnNm
    
    jsonData = getJson(url)

    try: #request에 성공한 station만을 추출하여 json 불러옴
      df = pd.json_normalize(jsonData['realtimeArrivalList'])
      df = df[(df['arvlCd'] == '1') & ((df['subwayId'] == '1001') | (df['subwayId'] == '1002'))] 
      #print(statnNm)
    except:
      print("Excepted: ", statnNm)
      continue

    #currTime = datetime.datetime.now(gettz("Asia/Seoul"))  #한국 시간대 사용
    df = pd.DataFrame(df)

    saveData(df, statnNm) #The data is saved
  

In [ ]:
while True:
  main()